In [2]:
import logging
from datetime import datetime
from typing import Dict, List, Any, Tuple
from rich.pretty import pprint
from types import SimpleNamespace
from tqdm import tqdm
import os, sys

import gym
import babyai_text
import torch
from transformers import PreTrainedTokenizer, AutoTokenizer
from trl import (
    PPOConfig,
    PPOTrainer,
    AutoModelForCausalLMWithValueHead,
    create_reference_model,
)

In [3]:
args = {
        # Training config
        "model_id": "meta-llama/Llama-3.2-3B-Instruct",
        "env_id": "BabyAI-MixedTrainLocal-v0",
        "num_shared_layers": None,
        "num_steps_train": 2000,
        "num_envs": 4,
        "seed" : 30,
        # PPO config
        "batch_size": 4,
        "mini_batch_size": 4,
        # "gradient_accumulation_steps": 4, 
        "optimize_device_cache": True,
        "early_stopping": False,
        # Env config
        "consecutive_invalid_actions_allowed": 5,
        "invalid_action_penalty": -0.1,
        "max_steps_per_episode": 100,
        # Generation kwargs
        "max_new_tokens": 10,
        "do_sample": False,
        "temperature": 0.8,
        "top_k": 20,
        "top_p": 0.90,
        # PEFT config
        "use_peft": True,
        "lora_r": 32,
        "lora_alpha": 32,
        "lora_dropout": 0.05,
        "lora_bias": "none",
    }
args = SimpleNamespace(**args)  # same type as argparse would return

In [4]:
envs = []
for i in range(args.num_envs):
    env = gym.make(args.env_id)
    env.seed(100 * args.seed + i)
    envs.append(env)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(args.model_id, padding_side="left")
model = AutoModelForCausalLMWithValueHead.from_pretrained(args.model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 

In [ ]:
ref_model = create_reference_model(model, num_shared_layers=args.num_shared_layers)